In [2]:
import numpy as np
import pandas as pd
import math

## Step 1: Gather and Prepare Data

### Input Data

#### Heavy Machiney Data

In [3]:
BC = 300          # Battery capacity (Kwh)
r = BC * 0.25     # Current SOC based on the percentage of the battery capacity
D = BC - r        # Required energy demand
v = 40           # Charging speed (Kw)
print(f"Energy demand : {D} kWh") 

Energy demand : 225.0 kWh


#### Stations Data

In [4]:
dataset = pd.read_excel('C:\\Users\\ASUS\\Downloads\\archive\\Final_dataset(Cleaned_extra_stations).xlsx')
dataset
CS = dataset['capacity']             # Charging speed (Kw)
a = dataset['available']             # Availability
latitude = dataset['latitude']       # Latitudes of stations
longitude = dataset['longitude']     # Longitudes of stations

## Step 2: Define the rules

### Distance Check

In [14]:
from geopy.distance import geodesic

# Ensure your dataset has the required columns
# Columns: 'name', 'address', 'latitude', 'longitude', 'cost_per_unit', 'available'

# Reference coordinate (latitude, longitude)
reference_point = (28.597250, 77.226350)

# Calculate distances using Vincenty formula (via geopy)
dataset['Distance'] = dataset.apply(lambda row: round(geodesic(reference_point, 
                                                               (row['latitude'], row['longitude'])).kilometers, 2), axis=1)

# Filter stations by distance condition (within 3 km)
results_distance = []
for _, row in dataset[dataset['Distance'] <= 3].iterrows():
    results_distance.append([row['name'], row['address'], 
                             row['latitude'], row['longitude'], row['cost_per_unit'], 
                             row['Distance'], row.get('available', None)])

# Convert to dataframe
filtered_distance_df = pd.DataFrame(results_distance, 
                                    columns=['Name', 'Address', 'Latitude', 'Longitude', 
                                             'Cost_per_Unit', 'Distance', 'Available'])

# View the filtered dataframe
filtered_distance_df.head()


,Name,Address,Latitude,Longitude,Cost_per_Unit,Distance,Available
0,REIL,Nizamuddin Railway station,28.588991,77.253240,11.56,2.79,0
1,EESL,"Outside Bharat Sanchar Bhawan, Ashoke Road, Ja...",28.621724,77.219286,10.00,2.80,0
2,EESL,"Palika Maternity Hospital, Block 11, Lodhi Col...",28.583518,77.222057,10.00,1.58,0
3,EESL,Outside Chelmsford Club/ Opposite CSIR Buildin...,28.618177,77.212590,9.50,2.68,2
4,EESL,"Press Club of India, 1, Raisina Road, Windsor ...",28.617517,77.213919,9.50,2.55,0


#### Availability Check

In [15]:

results_available = []
for _, row in dataset[dataset['available'] > 0].iterrows():                     # Filter stations for their availability
    results_available.append([row['name'], row['address'], row['latitude'], row['longitude'], row['cost_per_unit'], row.get('Distance', None), row['available']])


filtered_available_df = pd.DataFrame(results_available,                        # Convert to dataframe
                                     columns=['Name', 'Address', 'Latitude', 'Longitude', 'Cost_per_Unit', 'Distance', 'Available'])


filtered_available_df.head()

,Name,Address,Latitude,Longitude,Cost_per_Unit,Distance,Available
0,EESL,"NDMC Parking, Outside Jain Bhawan, Shaheed B...",28.633614,77.207691,10.0,4.42,1
1,EESL,"PSOI Club, Chanakyapuri, New Delhi 110021",28.586638,77.193352,9.5,3.44,1
2,EESL,Outside Chelmsford Club/ Opposite CSIR Buildin...,28.618177,77.212590,9.5,2.68,2
3,EESL,"Opposite ICICI Bank, Janpath, New Delhi",28.628528,77.219460,9.5,3.53,1
4,EESL,Talkatora Stadium Presidents Estate New Delhi ...,28.625753,77.194530,9.5,4.43,1


#### Combine two conditions

In [16]:
results_combined = []
for _, row in dataset[(dataset['Distance'] <= 3) & (dataset['available'] > 0)].iterrows(): # Combine both two conditions
    results_combined.append([row['name'], row['address'], row['latitude'], row['longitude'], row['cost_per_unit'], row['Distance'], row['available']])


filtered_combined_df = pd.DataFrame(results_combined,                                       
                                    columns=['Name', 'Address', 'Latitude', 'Longitude', 'Cost_per_Unit', 'Distance', 'Available'])

filtered_combined_df.head()

,Name,Address,Latitude,Longitude,Cost_per_Unit,Distance,Available
0,EESL,Outside Chelmsford Club/ Opposite CSIR Buildin...,28.618177,77.212590,9.5,2.68,2
1,REVOS,"E -12, ground floor, Lajpat Nagar 1, New Delhi...",28.573348,77.237225,10.0,2.85,1
2,REVOS,"E -12, ground floor, Lajpat Nagar 1, New Delhi...",28.573348,77.237225,10.0,2.85,1
3,REVOS,B215 lagpat nagar delhi,28.574852,77.241118,10.0,2.87,1
4,REVOS,B 215 Lajpat Nagar Delhi,28.575134,77.238969,10.0,2.74,1


In [17]:
filtered_combined_df

,Name,Address,Latitude,Longitude,Cost_per_Unit,Distance,Available
0,EESL,Outside Chelmsford Club/ Opposite CSIR Buildin...,28.618177,77.212590,9.5,2.68,2
1,REVOS,"E -12, ground floor, Lajpat Nagar 1, New Delhi...",28.573348,77.237225,10.0,2.85,1
2,REVOS,"E -12, ground floor, Lajpat Nagar 1, New Delhi...",28.573348,77.237225,10.0,2.85,1
3,REVOS,B215 lagpat nagar delhi,28.574852,77.241118,10.0,2.87,1
4,REVOS,B 215 Lajpat Nagar Delhi,28.575134,77.238969,10.0,2.74,1
5,REVOS,prem nagar near tyagraj stadium tyagraj park,28.579142,77.215722,10.0,2.26,1
6,REVOS,near mother dairy nizamuddin railway station,28.588216,77.255175,10.0,2.99,1
7,REVOS,a 111 3 sukhdev makrket kotla Mubarakpur,28.574880,77.227579,10.0,2.48,1
8,REVOS,"A753/1, sukhdev Market, Kotla Mubarakpur",28.574847,77.227747,10.0,2.49,1
9,REVOS,808 Sukh Dev market kotla Mubarakpur new delhi,28.574992,77.228454,10.0,2.48,1


#### Show the number of acceptable stations

In [18]:
filtered_distance_df.shape
print(f"The number of accepted stations: {len(filtered_distance_df)} ")

The number of accepted stations: 37 


#### Display the stations on map

In [19]:
import plotly.graph_objects as go

# Reference point coordinates
reference_point = {"latitude": reference_point[0], "longitude": reference_point[1], "name": "Reference Point"}

# Create the map figure
fig = go.Figure()

# Add the reference point as a larger marker
fig.add_trace(go.Scattermapbox(
    lat=[reference_point['latitude']],
    lon=[reference_point['longitude']],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=15,  # Larger size for reference point
        color='red'  # Distinct color
    ),
    name=reference_point['name'],  # Legend name
    hoverinfo='text',
    text=f"<b>{reference_point['name']}</b>"  # Tooltip text
))

# Add the chosen stations as smaller markers
fig.add_trace(go.Scattermapbox(
    lat=filtered_combined_df['Latitude'],  # Replace with your latitude column name
    lon=filtered_combined_df['Longitude'],  # Replace with your longitude column name
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=10,  # Smaller size for chosen stations
        color='blue'
    ),
    name="Stations",  # Legend name
    hoverinfo='text',
    text=filtered_combined_df['Name'] + "<br>" + filtered_combined_df['Address']  # Tooltip text
))

# Update the layout for the map
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=reference_point['latitude'], lon=reference_point['longitude']),
        zoom=12
    ),
    title="Map of Chosen Stations and Reference Point",
    title_x=0.5,  # Center the title
    height=600
)

# Show the map
fig.show()


#### Build the model

In [20]:
import heapq

t = 100                                                                              # Weight for distance (Euro)
p = filtered_combined_df['Cost_per_Unit']
# Step 1: Calculate the total cost dynamically
filtered_combined_df['Total Cost'] = (D * filtered_combined_df['Cost_per_Unit']) + (t * filtered_combined_df['Distance'])

# Step 2: Check required columns and find the top 5 cheapest stations
if 'Total Cost' in filtered_combined_df.columns:
    top_5 = heapq.nsmallest(
        5,
        filtered_combined_df[['Name', 'Address', 'Total Cost', 'Distance', 'Cost_per_Unit', 'Available']].itertuples(index=False),
        key=lambda x: x[2]  # Sorting by 'Total Cost' (2nd index in the tuple)
    )

    # Step 3: Convert to DataFrame
    cheapest_stations_df = pd.DataFrame(top_5, columns=['Name', 'Address', 'Total Cost', 'Distance', 'Cost_per_Unit', 'Available'])

    # Step 4: Display the results
    print("Top 5 Cheapest Stations in order:")

else:
    print("Error: 'Total Cost' column is missing.")

cheapest_stations_df.head()

Top 5 Cheapest Stations in order:


,Name,Address,Total Cost,Distance,Cost_per_Unit,Available
0,EESL Khan Market,"NDMC Parking, Khan Market, New Delhi, 110003",2284.0,0.34,10.0,2
1,EESL Prithviraj Mark,"NDMC Parking, Prithviraj Market, Prithviraj Ln...",2289.0,0.39,10.0,1
2,EESL Lodhi Garden,"NDMC Parking, Gate No. 1, Lodhi Gardens, Lodh...",2348.0,0.98,10.0,1
3,EESL Oriental Bank,"NDMC Parking, Outside Oriental Bank, Radial R...",2358.0,1.08,10.0,1
4,EESL Hotel Claridges,"NDMC Parking, Side of Hotel Claridges, Tees ...",2380.0,1.30,10.0,1


#### Display 5 best stations on map

In [21]:
# Coordinates for the 5 cheapest stations
cheapest_stations = cheapest_stations_df  # Assuming this contains 'Address' and 'Total Cost'
latitudes = filtered_combined_df.loc[cheapest_stations_df.index, 'Latitude']
longitudes = filtered_combined_df.loc[cheapest_stations_df.index, 'Longitude']

# Create the map figure
fig = go.Figure()

# Add the reference point
fig.add_trace(go.Scattermapbox(
    lat=[reference_point['latitude']],  # Use key 'latitude'
    lon=[reference_point['longitude']],  # Use key 'longitude'
    mode='markers+text',
    marker=dict(size=15, color='red'),
    text=["Reference Point"],
    textposition="top right",
    name="Reference Point"
))
# Add the 5 cheapest stations
fig.add_trace(go.Scattermapbox(
    lat=latitudes,
    lon=longitudes,
    mode='markers+text',
    marker=dict(size=12, color='blue'),
    text=[
        f"Name: {row['Name']}<br>"
        f"Address: {row['Address']}<br>"
        f"Distance: {row['Distance']} km<br>"
        f"Cost per Kwh: {row['Cost_per_Unit']} <br>"
        f"Total Cost: {row['Total Cost']}"
        for _, row in cheapest_stations.iterrows()
    ],
    textposition="bottom right",
    name="Cheapest Stations"
))


# Update layout with map details
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center=dict(
            lat=reference_point['latitude'], 
            lon=reference_point['longitude']
        ),
        zoom=12
    ),
    title="Top 5 Cheapest Stations and Reference Point",
    title_x=0.5,
    height=600
)


# Show the map
fig.show()